In [ ]:
import os
from enum import Enum
import dataclasses
import pathlib
import re
import requests
from requests.models import Response
import tqdm
import pandas as pd
import urllib
import dotenv

dotenv.load_dotenv()

class StatId(Enum):
    population: str = "0000010101"
    environment: str = "0000010102"
    economics: str = "0000010103"
    administration: str = "0000010104"
    education: str = "0000010105"
    labour: str = "0000010106"
    culture: str = "0000010107"
    housing: str = "0000010108"
    medical_care: str = "0000010109"
    social_security: str = "0000010110"
    household_finance: str = "0000010111"
    daily_routine: str = "0000010112"


@dataclasses.dataclass
class ClassData:
    id: str
    name: str
    data: pd.DataFrame


@dataclasses.dataclass
class StatsData:
    response: Response

    def get_values(self) -> pd.DataFrame:
        return pd.DataFrame(self.response.json()["GET_STATS_DATA"]["STATISTICAL_DATA"]["DATA_INF"]["VALUE"])

    def get_class(self) -> list[ClassData]:
        class_list: list[ClassData] = []
        class_obj = self.response.json()["GET_STATS_DATA"]["STATISTICAL_DATA"]["CLASS_INF"]["CLASS_OBJ"]
        for c in class_obj:
            data = c["CLASS"]
            class_list.append(
                ClassData(
                    id=c["@id"],
                    name=c["@name"],
                    data=pd.DataFrame(data if isinstance(data, list) else [data]),
                )
            )
        return class_list

    def output_feature_list(self):
        p = pathlib.Path("./feature_list.txt")
        p.touch()
        with p.open("w", encoding="utf-8") as f:
            for stat in self.StatId:
                f.write(stat.name + "\n")
                data = self.fetch(stat.value)
                for cls in self.extract_classes(data):
                    if cls.id == "cat01":
                        for code, feature in zip(cls.data["@code"], cls.data["@name"]):
                            formatted_feature = feature.replace(code + "_", "")
                            f.write(f"\t - {code:<10} | {formatted_feature}\n")
                f.write("\n")
                print("done.")

In [ ]:
def fetch(endpoint: str, params: dict[str, str]):
    base_url: str = f"https://api.e-stat.go.jp/rest/{os.getenv('API_VERSION')}/app/json/"
    url = urllib.parse.urljoin(base_url, endpoint)
    return requests.get(url, params=params)

In [ ]:
endpoint: str = "getStatsData"
params: dict[str, str] = {
    "appId": os.getenv("APP_ID"),
    "statsDataId": StatId.population.value,
}
stats_res = fetch(endpoint, params)

In [ ]:
stats_data = StatsData(stats_res)
stats_data.get_values().head()

In [ ]:
[i.id for i in stats_data.get_class()]

In [ ]:
for i in stats_data.get_class():
    if i.id == "cat01":
        for j in i.data.sort_values("@code")["@name"]:
            print(j)

In [ ]:
def extract_num_from_code(code: str) -> int:
    match_obj = re.search(r"A([0-9]*)", code)
    if match_obj:
        return int(match_obj.group(1))

for i in stats_data.get_class():
    if i.id == "cat01":
        cat01_df = i.data

def query_by_cat01(cat01: str):
    endpoint: str = "getStatsData"
    params: dict[str, str] = {
        "appId": os.getenv("APP_ID"),
        "statsDataId": StatId.population.value,
        "cdCat01": i
    }
    return fetch(endpoint, params)

p = pathlib.Path("./data/population")
p.mkdir(exist_ok=True, parents = True)

cat01_df["num_in_code"] = cat01_df["@code"].apply(lambda x: extract_num_from_code(x))
population_age_gender_codes = cat01_df[cat01_df["num_in_code"].between(120101, 122102)]["@code"].to_list()
for i in tqdm.tqdm(population_age_gender_codes):
    res = query_by_cat01(i)
    i_stats_data = StatsData(res)
    i_stats_data.get_values().to_csv(p / f"{i}.csv", index=False)